# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.

## Setup

In [40]:
from pathlib import Path
from freqtrade.configuration import Configuration

from freqtrade.data.dataprovider import DataProvider
from freqtrade.plugins.pairlistmanager import PairListManager
from freqtrade.exchange import Exchange
from freqtrade.resolvers import ExchangeResolver, StrategyResolver

import nest_asyncio
nest_asyncio.apply()


import os
#print("Current working directory: {0}".format(os.getcwd()))
os.chdir('/freqtrade/')
#print("Current working directory: {0}".format(os.getcwd()))


#from pathlib import Path
#my_file = Path("user_data/all_config/05_config_volume_100_static.json")
#if my_file.is_file():
#    print('config file exists')
#else:
#    print("config does not exist")

# config = Configuration.from_files([])
config = Configuration.from_files(["/freqtrade/user_data/all_config/05_config_volume_100_static.json"])

ft_exchange = ExchangeResolver.load_exchange(
    config['exchange']['name'], 
    config=config, 
    validate=True
)

ft_pairlists = PairListManager(ft_exchange, config)
ft_dataprovider = DataProvider(config, ft_exchange, ft_pairlists)


# Define some constants
config["timeframe"] = "5m"
# Name of the strategy class
config["strategy"] = "NostalgiaForInfinityV3"
# Location of the data
data_location = Path(config['user_data_dir'], 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC/USDT"

In [41]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "json",
                            )

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
candles.head()

Loaded 147685 rows of data for BTC/USDT from /freqtrade/user_data/data/binance


,date,open,high,low,close,volume
0,2020-01-01 00:00:00+00:00,7195.24,7196.25,7178.64,7179.78,95.509133
1,2020-01-01 00:05:00+00:00,7179.76,7191.77,7178.20,7191.07,59.365225
2,2020-01-01 00:10:00+00:00,7193.15,7193.53,7180.24,7180.97,48.068510
3,2020-01-01 00:15:00+00:00,7180.97,7186.40,7177.35,7178.29,32.192929
4,2020-01-01 00:20:00+00:00,7177.71,7182.46,7175.47,7176.96,49.027397


## Load and run strategy
* Rerun each time the strategy file is changed

In [42]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)
strategy.dp = ft_dataprovider

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {'pair': pair})
df.tail()

,date,open,high,low,close,volume,date_1h,open_1h,high_1h,low_1h,...,lips,smma_lips,teeth,smma_teeth,jaw,smma_jaw,volume_mean_4,safe_dips,buy,sell
147680,2021-05-27 18:40:00+00:00,38991.68,39125.69,38973.44,39122.98,140.886654,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39032.418,39032.393333,39048.66875,39057.36725,39067.070000,39061.338846,160.424603,True,NaN,NaN
147681,2021-05-27 18:45:00+00:00,39120.01,39222.13,39033.41,39120.02,164.994399,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39055.926,39036.366667,39056.81625,39052.23375,39072.269231,39064.964038,165.987505,True,NaN,NaN
147682,2021-05-27 18:50:00+00:00,39120.02,39159.22,39040.55,39048.86,132.023996,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39072.674,39053.672667,39049.45500,39049.03475,39068.228462,39065.926731,163.960794,True,NaN,NaN
147683,2021-05-27 18:55:00+00:00,39048.86,39115.38,38970.97,39089.57,120.840707,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39074.586,39067.728667,39052.56750,39050.05625,39057.893077,39064.153846,143.487786,True,NaN,NaN
147684,2021-05-27 19:00:00+00:00,39094.18,39112.00,38852.16,38885.24,246.205988,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39053.334,39066.864667,39037.91250,39049.08400,39046.835385,39062.236731,139.686439,True,NaN,NaN


### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [43]:
# Report results
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

Generated 68.0 buy signals


,date,open,high,low,close,volume,date_1h,open_1h,high_1h,low_1h,...,lips,smma_lips,teeth,smma_teeth,jaw,smma_jaw,volume_mean_4,safe_dips,buy,sell
date,,,,,,,,,,,,,,,,,,,,,
2021-05-27 18:40:00+00:00,2021-05-27 18:40:00+00:00,38991.68,39125.69,38973.44,39122.98,140.886654,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39032.418,39032.393333,39048.66875,39057.36725,39067.070000,39061.338846,160.424603,True,NaN,NaN
2021-05-27 18:45:00+00:00,2021-05-27 18:45:00+00:00,39120.01,39222.13,39033.41,39120.02,164.994399,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39055.926,39036.366667,39056.81625,39052.23375,39072.269231,39064.964038,165.987505,True,NaN,NaN
2021-05-27 18:50:00+00:00,2021-05-27 18:50:00+00:00,39120.02,39159.22,39040.55,39048.86,132.023996,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39072.674,39053.672667,39049.45500,39049.03475,39068.228462,39065.926731,163.960794,True,NaN,NaN
2021-05-27 18:55:00+00:00,2021-05-27 18:55:00+00:00,39048.86,39115.38,38970.97,39089.57,120.840707,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39074.586,39067.728667,39052.56750,39050.05625,39057.893077,39064.153846,143.487786,True,NaN,NaN
2021-05-27 19:00:00+00:00,2021-05-27 19:00:00+00:00,39094.18,39112.00,38852.16,38885.24,246.205988,2021-05-27 17:00:00+00:00,38822.28,39250.0,38691.0,...,39053.334,39066.864667,39037.91250,39049.08400,39046.835385,39062.236731,139.686439,True,NaN,NaN


## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [44]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file 
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"

In [52]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'NostalgiaForInfinityV3'
#print(stats['strategy'][''])
# print(stats['strategy']['NostalgiaForInfinityV3'].keys())

# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


[{'key': 'MATIC/USDT', 'trades': 14, 'profit_mean': 0.03489245357142857, 'profit_mean_pct': 3.489245357142857, 'profit_sum': 0.48849435, 'profit_sum_pct': 48.85, 'profit_total_abs': 903875.4055833, 'profit_total': 18.077508111666, 'profit_total_pct': 1807.75, 'duration_avg': '0:53:00', 'wins': 14, 'draws': 0, 'losses': 0}, {'key': 'OGN/USDT', 'trades': 4, 'profit_mean': 0.011921805, 'profit_mean_pct': 1.1921805, 'profit_sum': 0.04768722, 'profit_sum_pct': 4.77, 'profit_total_abs': 662927.75701544, 'profit_total': 13.2585551403088, 'profit_total_pct': 1325.86, 'duration_avg': '6:45:00', 'wins': 2, 'draws': 0, 'losses': 2}, {'key': 'DOGE/USDT', 'trades': 19, 'profit_mean': 0.0794142052631579, 'profit_mean_pct': 7.94142052631579, 'profit_sum': 1.5088699, 'profit_sum_pct': 150.89, 'profit_total_abs': 583320.68783751, 'profit_total': 11.6664137567502, 'profit_total_pct': 1166.64, 'duration_avg': '3:26:00', 'wins': 18, 'draws': 0, 'losses': 1}, {'key': 'ETC/USDT', 'trades': 6, 'profit_mean':

In [53]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

pair       sell_reason      
AAVE/USDT  target_profit_u_1    2
           sell_signal          1
ADA/USDT   target_profit_u_1    2
           sell_signal          1
           target_profit_2      1
                               ..
WIN/USDT   target_profit_u_2    1
XLM/USDT   target_profit_1      1
XRP/USDT   target_profit_u_1    2
           target_profit_u_2    1
YFI/USDT   target_profit_u_1    1
Name: sell_reason, Length: 63, dtype: int64

### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [54]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["sell_reason"].value_counts()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 1

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,bin_size=0.01)
fig.show()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.